<a href="https://colab.research.google.com/github/LarionovDaniil/vk_handshakes/blob/main/friends_or_not_vk_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from threading import Thread
import requests
import time
import numpy as np
import math

In [ ]:
version = '5.131'

# Для работы программы необходимо сгенерировать 5 ключей доступа пользователя и 5 сервисных ключей доступа.

token_user = '---'
token_user2 = '---'
token_user3 = '---'
token_user4 = '---'
token_user5 = '---'

token_service = '---'
token_service2 = '---'
token_service3 = '---'
token_service4 = '---'
token_service5 = '---'

In [ ]:
connections = []
rez = [[],[],[],[],[]]
num_of_tokens = 5
token_users = [token_user, token_user2, token_user3, token_user4, token_user5]
tokens_service = [token_service, token_service2, token_service3, token_service4, token_service5]
connections_limit = 11
delay = 0.1 # задержка зависит от скорости интернета(больше скорость -> больше задержка нужна)

In [ ]:
def id_from_domain(domain:str) -> int:
    '''
    Returns id from url/domain or code of error.

            Parameters:
                    domain (string): can be url or domain

            Returns:
                    user id if id exists and page is open
                    -1 if id doesn't exist
                    -2 if page deleted
                    -3 if page is closed
    '''

    url = 'https://api.vk.com/method/users.get'

    response = requests.get(url,
                            params={
                                'access_token': token_service,
                                'v': version,
                                'user_id': domain
                            }
                            )
    page_info = response.json()['response']
    if page_info == []:
        return -1
    elif 'deactivated' in page_info[0]:
        return -2
    elif page_info[0]['is_closed'] == True:
        return -3
    return page_info[0]['id']

In [ ]:
def returns_meaning(num:int, id:str) -> int:
    '''Returns meaning of code errors'''
    errors = {
        -1: 'Не удалось определить ID',
        -2: 'Cтраница удалена',
        -3: 'Cтраница закрыта'
    }
    print(errors[num], id)
    return 0

In [ ]:
def user_url_re(url_input:str) -> int:
    '''Get user id.

        Call id_from_domain to get user id if user taped smth different.

        https://vk.com/id92273240 -> 92273240
        https://vk.com/danya.larionov -> 92273240
        danya.larionov -> 92273240

        Parameters:
                    url_input: tapped url

        Returns:
                    user id
    '''

    url = url_input

    if url.isdigit():
        id = id_from_domain(url)
    else:
        com = url.find("com/")
        if com != -1:
            id = id_from_domain(url[com + 4:])
        else:
            id = id_from_domain(url)
    return id

In [ ]:
def get_users_info(user_id:int, token:str) -> tuple[str, str, str]:
    '''Get information about user

            Parameters:
                    user_id: user id
                    token: user or service token
            Returns:
                    extended user information.
    '''

    url = 'https://api.vk.com/method/users.get'

    response = requests.get(url,
                            params={
                                'access_token': token,
                                'v': version,
                                'user_id': user_id,
                                'fields':'photo_400_orig,photo_max_orig'
                            }
                            )

    data = response.json()['response'][0]
    url = 'https://vk.com/id'+str(data['id'])
    photo = data['photo_400_orig'] if 'photo_400_orig' in data else \
                                            data['photo_max_orig']
    full_name = ' '.join((data['first_name'], data['last_name']))
    time.sleep(delay)
    return url, photo, full_name

In [ ]:
def get_users_friends(user_id:int, privacy:str='open') -> np.ndarray:
    '''Returns slist of friends.

            Parameters:
                    user_id: user id(only number)
                    privacy: open, close

            Returns:
                    list of friends with open pages if privacy == open
                                        else list of all friends
    '''

    url = 'https://api.vk.com/method/friends.get'

    response = requests.get(url,
                            params={
                                'access_token': token_service2,
                                'v': version,
                                'user_id': user_id,
                                'fields':'is_closed'
                            }
                            )
    data = response.json()

    friends = data['response']['items']

    if len(friends) == 0:
        print(f'У {get_users_info(user_id, token_service)[2]} '
              f'нет друзей')
        return np.array([])

    elif len(friends) > 1000:
        print(f'https://vk.com/id{user_id} очень '
              f'дружелюбный(-ая), '
              f'поиск может занять больше времени')

    if privacy == 'open':
        friends_open = np.array([friend['id'] for friend in friends
                        if not friend['is_closed']
                        and 'deactivated' not in friend.keys()])
        return friends_open
    else:
        return np.array(friends)

In [ ]:
def mutual_friends_one_to_many(user_1:int,
                               users_2_friends:np.ndarray,
                               token:str) -> np.ndarray:
    '''Get mutual friends

            Parameters:
                    user_1: first user id
                    users_2_friends: list of second user's friends
                    token: only user token
            Returns:
                    list of IDs of mutual friends between a pair of users.
    '''

    url = 'https://api.vk.com/method/friends.getMutual'

    response = requests.get(url,
                            params={
                                'access_token': token,
                                'v': version,
                                'source_uid': user_1,
                                'target_uids': ", ".join(map(str, users_2_friends))
                            }
                            )
    data = response.json()
    time.sleep(delay+0.1)
    return np.array(data['response'])

In [ ]:
def output(list_ids:list, token_i:str) -> int:
    '''Add connection to list connections

            Parameters:
                    list_ids: first user id
                    token_i: user or service token
            Returns:
                    0
    '''

    global connections

    info = []
    for id in list_ids:
        info.append(get_users_info(id, token_i))

    # urls = [url[0] for url in info]
    # picture = [pic[1] for pic in info]
    names = [name[2] for name in info]

    res = ' -> '.join(names)
    connections.append(res)
    return 0

In [ ]:
def output_thread(list_ids:list[list], token_i:str) -> int:
    '''Add connection to list connections, using for threading

            Parameters:
                    list_ids: list of lists with id connections
                    token_i: user or service token
            Returns:
                    0
    '''

    global connections

    for ids in list_ids:
        info = []
        for id in ids:
            info.append(get_users_info(id, token_i))
            # urls = [url[0] for url in info]
            # picture = [pic[1] for pic in info]
        names = [name[2] for name in info]

        res = ' -> '.join(names)
        connections.append(res)

    return 0

In [ ]:
def two_handshakes(id_first:int, id_second:int,
                   friends_first:np.ndarray,
                   friends_second:np.ndarray) -> int:
    '''Check if users know each other through one person

            Parameters:
                    id_first: first user id
                    id_second: second user id
                    friends_first: array of first user's friends
                    friends_second: array of second user's friends
            Returns:
                    0 if yes else 1
    '''
    two_hand = friends_first[np.in1d(friends_first, friends_second)]

    if len(two_hand) > 0:
        for id in two_hand:
            output([id_first, id, id_second], token_service4)
        return 0
    return 1

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
def part_parsing3(id_1, friends_part, id_2,
                  user_token, service_token, first_less) -> int:
    '''Call output for connection of ids

            Parameters:
                    id_1: first user id
                    friends_part: part for getMutual method
                    id_2: second user id
                    user_token: user token
                    service_token: service token
                    first_less: True if first user has
                                    less friends else False
            Returns:
                    0
    '''
    id_main = id_2 if first_less else id_1

    for friends in friends_part:
        mutal_result = mutual_friends_one_to_many(id_main, friends, user_token)
        common = list(map(lambda x: x['common_count'] != 0, mutal_result))
        if len(mutal_result[common]) > 0:
            for comm in mutal_result[common]:
                for id_comm in comm['common_friends']:
                    if len(connections) < connections_limit:
                        if first_less:
                            output([id_1, comm['id'], id_comm, id_2],
                                   service_token)
                        else:
                            output([id_1, id_comm, comm['id'], id_2],
                                   service_token)
    return 0

In [ ]:
def make_splits(friends_for_split:np.ndarray) -> list:
    '''getMutal has limit of 100 people that's why I split friends
            Parameters:
                    friends_for_split:
            Returns:
                    array of arrays with splitted friends
    '''
    length_friends = len(friends_for_split)
    parts = math.ceil(length_friends/100)
    splits = np.array_split(friends_for_split, parts)
    less_hundred_splits = np.array_split(friends_for_split, num_of_tokens)
    five_splits = np.array_split(splits, num_of_tokens) \
                    if length_friends > 100 else \
                    np.array_split(less_hundred_splits, num_of_tokens)
    return five_splits

In [41]:
def three_handshakes(id_first, id_second, friends_first, friends_second):
    '''Check if users know each other through two persons

            Parameters:
                    id_first: first user id
                    id_second: second user id
                    friends_first: array of first user's friends
                    friends_second: array of second user's friends
            Returns:
                    0 if yes else 1
    '''
    first_less = len(friends_first) < len(friends_second)

    friends_to_split = friends_first if first_less else friends_second
    five_splits = make_splits(friends_to_split)

    thread = {}

    for i in range(num_of_tokens):
        thread[i] = Thread(target=part_parsing3, args=(id_first,
                                                    five_splits[i],
                                                    id_second,
                                                    token_users[i],
                                                    tokens_service[i],
                                                    first_less,))
        thread[i].start()
    for i in range(num_of_tokens):
        thread[i].join()

    if len(connections) > 0:
        return 0
    return 1

In [ ]:
def part_parsing4(id_1, friends_first, friends_part, id_2, i):
    '''Call output_thread for connection of ids

            Parameters:
                    id_1: first user id
                    friends_first: array of first user's friends
                    friends_part: part for getMutual method
                    id_2: second user id
                    token: user token
                    i: for rez index
            Returns:
                    0
    '''
    global rez

    for friend1 in friends_first:
        for friends2 in friends_part:
            commons = mutual_friends_one_to_many(friend1, friends2, token_users[i])
            common = list(map(lambda x: x['common_count'] != 0, commons))
            if len(commons[common]) > 0:
                for id in commons[common]:
                    for friend2 in id['common_friends']:
                        if len(connections) < connections_limit:
                            rez[i].append([id_1, friend1, friend2,
                                       id['id'], id_2])
    return 0

In [ ]:
def four_handshakes(id_first, id_second, friends_first, friends_second):
    '''Check if users know each other through three persons

            Parameters:
                    id_first: first user id
                    id_second: second user id
                    friends_first: array of first user's friends
                    friends_second: array of second user's friends
            Returns:
                    0 if yes else 1
    '''
    five_splits =  make_splits(friends_second)
    thread = {}

    for i in range(num_of_tokens):
      thread[i] = Thread(target=part_parsing4, args=(id_first,
                                                     friends_first,
                                                     five_splits[i],
                                                     id_second,
                                                     i,))
      thread[i].start()
    for i in range(num_of_tokens):
      thread[i].join()

    for i in range(len(rez)):
      thread[i] = Thread(target=output_thread, args=(rez[i],
                                                     tokens_service[i],))
      thread[i].start()
    for i in range(len(rez)):
      thread[i].join()

    if len(connections) > 0:
        return 0
    return 1

In [ ]:
def connection(id1, id2):
    '''get user's friends if profiles are open, check if users
        know each other and call two_handshakes, three_handshakes,
                                                    four_handshakes.
        print connections or says that there aren't any connections
                                                            between users

            Parameters:
                    id1: first user id
                    id2: second user id
            Returns:
                    0
    '''

    global connections, rez
    connections = []
    rez = [[],[],[],[],[]]

    friends1 = get_users_friends(id1)
    if id2 in friends1:
        output([id1, id2], token_user)
    else:
        friends2 = get_users_friends(id2)
        if two_handshakes(id1, id2, friends1, friends2):
            if three_handshakes(id1, id2, friends1, friends2):
                if four_handshakes(id1, id2, friends1, friends2):
                    print('Нет цепочки рукопожатий')

    if len(connections) > 0:
        print(*connections, sep='\n')
    return 0

In [ ]:
def init(id1:str, id2:str='leonid_agutin') -> int:
    '''Main function check ids by calling user_url_re and call connection

            Parameters:
                    id1: first user id
                    id2: second user id
            Returns:
                    1 if smth wrong with first id
                    2 - with second id
                    else 0
    '''
    id1_num = user_url_re(id1)
    if id1_num < 0:
        returns_meaning(id1_num, id1)
        return 1

    id2_num = user_url_re(id2)
    if id2_num < 0:
        returns_meaning(id2_num, id2)
        return 2

    else:
        connection(id1_num, id2_num)
    return 0

In [ ]:
init('danya.larionov', 'dm') # Время выполнения - 28 секунд.

Daniil Larionov -> Maria Larionova -> Ildar Belyalov -> Zera Chereshneva -> Dmitry Medvedev
Daniil Larionov -> Sergey Yakubeyko -> Irina Makukhina -> Ivan Zasursky -> Dmitry Medvedev
Daniil Larionov -> Sergey Yakubeyko -> Viktor Buleyko -> Vladimir Burmatov -> Dmitry Medvedev
Daniil Larionov -> Sergey Yakubeyko -> Denis Chernenko -> Vladimir Burmatov -> Dmitry Medvedev
Daniil Larionov -> Danya Shaydayuk -> Dmitry Shestakov -> Elena Bocherova -> Dmitry Medvedev
Daniil Larionov -> Olya Tsyplakova -> Yury Dedenyov -> Vladimir Burmatov -> Dmitry Medvedev
Daniil Larionov -> Seva Sezonov -> Sergey Shishov -> Maria Kozhevnikova -> Dmitry Medvedev


0

In [40]:
init('danya.larionov', '127943481') # Время выполнения - 11 секунд.

Daniil Larionov -> Denis Smagin -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Danila Strizhkov -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Vlad Tsypilev -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Alexander Semyonov -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Danila Turlyuk -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Kirill Shokurov -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Oleg Shevkun -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Vasily Zotov -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Semyon Shevchenko -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Vladislav Davydov -> Egor Badika -> Katya Agaeva
Daniil Larionov -> Alexander Inichkin -> Egor Badika -> Katya Agaeva


0